In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [79.8 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]m
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all

### Create a baseline to beat

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm  
test_raw = pd.read_csv('/kaggle/input/data00/test_raw.csv')
test_raw['Clinician'] = test_raw['Prompt'].apply(lambda x: "summary "+ " ".join(x.split('.')[1:]).replace('\n\n', ' ').replace('\n', ' '))
test_raw['Clinician'][0]

                                                 

'summary  A 24 year old female complains of sharp pain in the right side of the nose that started 2 days ago which has been gradually worsening  No past medical history  On assessment there is tenderness on palpation on the right side of the nasal bridge, no visible signs of inflammation or infection  Vitals: BP 129/81 mmHg, PR 81, RR 20, T 36 8, SPO2 94%  Question: What could be the diagnosis of the patient?'

In [12]:
#test_raw[['Master_Index', 'Clinician']].to_csv('baseline2.csv', index=False)

In [3]:
import threading
import subprocess
import time

def run_ollama_serve():
    process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    out, err = process.communicate(timeout=10)
    print("Output:", out)
    print("Error:", err)


thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)  

Exception in thread Thread-5 (run_ollama_serve):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_35/2783055524.py", line 11, in run_ollama_serve
  File "/usr/lib/python3.11/subprocess.py", line 1209, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 2116, in _communicate
    self._check_timeout(endtime, orig_timeout, stdout, stderr)
  File "/usr/lib/python3.11/subprocess.py", line 1253, in _check_timeout
    raise TimeoutExpired(
subprocess.TimeoutExpired: Command '['ollama', 'serve']' timed out after 10 seconds


In [4]:
#https://ollama.com/search
!ollama pull qwen2.5:0.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling c5396e06af29:   0% ▕                  ▏  58 KB/397 MB                  pulling manifest 
pulling c5396e06af29:   4% ▕                  ▏  17 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  20% ▕███               ▏  79 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  33% ▕█████             ▏ 129 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  39% ▕███████           ▏ 154 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  49% ▕████████          ▏ 193 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  58% ▕██████████        ▏ 232 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  62% ▕███████████       ▏ 245 MB/397 MB                  pulling manifest 
pulling c5396e06af29:  67% ▕████████████      ▏ 268 MB

In [5]:
!pip install langchain-ollama  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.23
    Uninstalling langsmith-0.3.23:
      Successfully uninstalled langsmith-0.3.23
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but yo

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_ollama.llms import OllamaLLM
# from IPython.display import Markdown

# template = """Question: {question}   

# Answer: Let's think step by step."""

# prompt = ChatPromptTemplate.from_template(template)

# # Load model
# model = OllamaLLM(model="qwen2.5:0.5b-instruct", max_tokens=250, temperature= 0.1,
#                  num_ctx = 1024 ,#reduce memory usage,
#                  keep_alive = '30m'
                 
#                  ).with_retry()

# chain = prompt | model

# display(Markdown(chain.invoke({"question": "What's the length of hypotenuse in a right angled triangle"})))

In [6]:
train_path = "/kaggle/input/data12/train.csv"
test_path = "/kaggle/input/data12/test.csv"

# NOW, read the CSV files into Pandas DataFrames
train = pd.read_csv(train_path)
test= pd.read_csv(test_path)

# Now you can use .head() on the DataFrames
display(train.head(), test.head()) 

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_VBWWP,uasin gishu,sub county hospitals and nursing homes,18.0,i am a nurse with 18 years of experience in ge...,pediatric emergency burns,surgery,summary a 4 year old with 5 superficial burns ...,given your vast experience as a nurse in uasin...,1 immediate treatment protocol for second degr...,here s a response addressing the questions reg...,288514009 burn involving 5 percent of body sur...
1,ID_XMBBY,uasin gishu,national referral hospitals,17.0,i am a nurse with 17 years of experience in ge...,child health,paediatrics,summary 6 year old present with vomiting and a...,clinical summary • a 6 year old girl with know...,based on the symptoms and signs you ve describ...,based on the presentation the 6 year old girl ...,420270002 ketoacidosis due to type 1 diabetes ...
2,ID_JZNZW,kiambu,sub county hospitals and nursing homes,12.0,i am a nurse with 12 years of experience in ge...,general emergency,internal medicine,summary a 47 year old man presents with severe...,in this case you re dealing with a 47 year old...,firstly i must commend you on your thorough hi...,this 47 year old male presenting with severe r...,13200003 peptic ulcer disorder 25458004 acute ...
3,ID_QOQTK,uasin gishu,national referral hospitals,12.0,i am a nurse with 12 years of experience in pr...,critical care,internal medicine,summary 72 year old female with inability to w...,given er s clinical presentation and vitals th...,to me with this query based on the information...,this 92 year old female patient er presents wi...,14760008 constipation finding 419284004 altere...
4,ID_ZFJBM,uasin gishu,national referral hospitals,16.0,i am a nurse with 16 years of experience in ge...,adult health,internal medicine,a 22 year old female presents with headache di...,the 22 year old female patient is presenting w...,thank you for presenting this case based on th...,this 22 year old female patient presents with ...,95874006 carbon monoxide poisoning from fire d...


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel
0,ID_CUAOY,uasin gishu,sub county hospitals and nursing homes,2.0,i am a nurse with 2 years of experience in gen...,adult health,surgery ent
1,ID_OGSAY,kiambu,sub county hospitals and nursing homes,22.0,i am a nurse with 22 years of experience in ge...,child health,surgery
2,ID_TYHSA,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,general emergency,internal medicine
3,ID_CZXLD,kakamega,dispensaries and private clinics,NaN,i am a nurse working in a dispensaries and pri...,child health,paediatrics
4,ID_ZJQUQ,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,child health,paediatrics


In [7]:
for idx, row in train.head(5).iterrows():
  print(row['Prompt'])
  print("-"* 200)
  print(row['Clinician'])
  print("*"* 200) 


i am a nurse with 18 years of experience in general nursing working in a sub county hospitals and nursing homes in uasin gishu county in kenya a 4 year old child presents to the emergency department with second degree burns on the forearm after accidentally touching a hot stove the child was playing in the kitchen when they reached out to touch the stove the burns cover about 5 of the total body surface area the child is alert and crying with redness blisters and swelling on the affected area the burns appear to be superficial to moderate in severity the child is in mild pain and there is no indication of airway or breathing distress no other injuries are noted questions 1 what is the immediate treatment protocol for second degree burns in paediatric patients 2 should any tetanus prophylaxis be considered in this case 3 what follow up care should be recommended for burn healing
-------------------------------------------------------------------------------------------------------------

In [8]:
import string

def clean_sentence(text: str) -> str:
    # Remove punctuation
    text_no_punct = text.translate(str.maketrans('', '', string.punctuation))
    # Remove newlines and carriage returns
    text_no_newlines = text_no_punct.replace('\n', ' ').replace('\r', ' ')
    # Convert to lowercase and strip extra whitespace
    return text_no_newlines.lower().strip() 



In [9]:
from textwrap import dedent
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
# Load model
model = OllamaLLM(model="qwen2.5:0.5b", max_tokens=250, temperature= 0.1,
                 num_ctx = 1024 ,#reduce memory usage, 
                 ).with_retry()

# Optimized prompt with enhanced structure and clinical tone
optimized_prompt_template = """
You are a Kenyan clinical officer. Given a medical scenario written by a nurse, your task is to:

1. Summarize the patient case using professional clinical language.
2. Provide a structured response with diagnosis, investigations, management, and follow-up.
3. Ensure the format and style matches the following examples exactly.

### Example Outputs:

Summary:
A 4-year-old with 5% superficial burns. No other injuries.

Immediate Management:
 * Paracetamol analgesics to ensure child has minimal or no pain
 * Cleaning and dressing of wound with silver sulfadiazine
 * Topical prophylactic considered in this case

Follow-up Care:
 * Good nutrition – high-protein diet

****************************************************************************************************

Summary:
6-year-old with vomiting and abdominal pain. Known diabetic on insulin but non-adherent due to lack of funds. Confused, Kussmaul breathing, fruity breath, dry tongue, blurred vision.Temp (normal), Pulse ↑120, RR ↑48, SpO₂ ↓90%.

Diagnosis:
Diabetic Ketoacidosis (DKA) due to insulin omission in a type 1 diabetic patient.

Immediate Management:
 * Insert IV line and administer normal saline bolus
 * Continuous insulin infusion (0.1 U/kg/hr)
 * Monitor blood glucose
 * Add potassium to IV fluids if hypokalemic
 * Treat underlying infection
 * Monitor ketones, pH, and bicarbonate
 
 Investigations:
 * Urinalysis
 * Blood gas analysis
 * Random blood sugar (RBS)
 * HbA1c
 * UECs
 * CBC
****************************************************************************************************

### INPUT:
{question}
### OUTPUT:
"""


prompt = ChatPromptTemplate.from_template(optimized_prompt_template)
chain = prompt | model

test['Clinician'] = ""
for idx, row in tqdm(test.iterrows(), total =len(test)):
  question = row['Prompt']
  # display(Markdown(chain.invoke({"question": question})))
  answer = chain.invoke({"question": question})
  processed_answer = clean_sentence(answer)
  # print(processed_answer)
  processed_answer = processed_answer.replace('\n\n', ' ').replace('\n', ' ').replace('**', '')
  test.loc[idx, 'Clinician'] = processed_answer


100%|██████████| 100/100 [44:28<00:00, 26.69s/it] 


In [10]:
test[['Prompt', 'Clinician']].head()

,Prompt,Clinician
0,i am a nurse with 2 years of experience in gen...,summary a 24yearold female with a history of d...
1,i am a nurse with 22 years of experience in ge...,summary a 2yearold with severe burns and respi...
2,i am a nurse working in a national referral ho...,summary a 22yearold man with a history of wea...
3,i am a nurse working in a dispensaries and pri...,summary a 6yearold girl with a history of twit...
4,i am a nurse working in a health centres in ka...,summary a 1yearold boy with severe skin rash ...


In [ ]:
''''for idx, row in test_raw.sample(5).iterrows():
  print(row['Prompt'])
  print("-"* 200)
  print(row['Clinician'])
  print("*"* 200)

''''

In [11]:
test[['Master_Index', 'Clinician']].to_csv("v6_qwen_train.csv", index=False)